<a href="https://colab.research.google.com/github/Mukta1901/PyAssistant/blob/main/demoMeetingSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [1]:
!pip install git+https://github.com/openai/whisper.git
# install the original whisper package from the OpenAI GitHub repository

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-mk_wjzmt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-mk_wjzmt
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import streamlit as st
import whisper
import requests
import os
import subprocess
import threading
import time
from transformers import BartForConditionalGeneration, BartTokenizer

# Initialize Whisper (Automatically downloads the model if not available)
model = whisper.load_model("small")

# Load BART-CNN Model & Tokenizer (Automatically downloads the model if not available)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
summarizer = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Directory for temp files
TEMP_DIR = "temp_videos"
os.makedirs(TEMP_DIR, exist_ok=True)

def download_video(video_url):
    """Download a video file from a direct URL."""
    local_filename = os.path.join(TEMP_DIR, "downloaded_video.mp4")
    response = requests.get(video_url, stream=True)
    if response.status_code == 200:
        with open(local_filename, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return local_filename
    return None

def extract_audio(video_path):
    """Extract audio from a video file using FFmpeg."""
    audio_path = video_path.replace(".mp4", ".mp3")
    command = ["ffmpeg", "-i", video_path, "-vn", "-acodec", "libmp3lame", "-b:a", "128k", audio_path, "-y"]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return audio_path

def transcribe_audio(audio_file, transcript_result):
    """Transcribe audio using Whisper."""
    start_time = time.time()
    transcription = model.transcribe(audio_file, language="en", condition_on_previous_text=False, fp16=False)
    end_time = time.time()
    transcript_result.append((transcription["text"], end_time - start_time))

def chunk_text(text, max_tokens=1024):
    """Splits long text into smaller chunks of <= max_tokens words."""
    words = text.split()
    return [" ".join(words[i:i + max_tokens]) for i in range(0, len(words), max_tokens)]

def transcribe_and_summarize(audio_file):
    """Transcribe and summarize large transcripts in chunks."""
    transcript_result = []
    transcription_thread = threading.Thread(target=transcribe_audio, args=(audio_file, transcript_result))
    transcription_thread.start()
    start_audio_time = time.time()
    audio_path = extract_audio(audio_file)
    end_audio_time = time.time()
    transcription_thread.join()
    transcript_text, transcription_time = transcript_result[0] if transcript_result else ("No transcription available.", 0)

    # Chunking transcript
    chunks = chunk_text(transcript_text, max_tokens=1024)
    summarized_chunks = []
    start_summary_time = time.time()

    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = summarizer.generate(inputs["input_ids"], max_length=300, min_length=100, num_beams=6, early_stopping=True)
        summarized_chunks.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))

    final_summary = " ".join(summarized_chunks)
    end_summary_time = time.time()
    return transcript_text, final_summary, transcription_time, (end_summary_time - start_summary_time), (end_audio_time - start_audio_time)

# Streamlit UI
st.title("🎥 Meeting Summarization App (BART-CNN)")
st.sidebar.header("Options")
video_url = st.sidebar.text_input("Enter Video URL (MP4, AVI, MKV):")
uploaded_file = st.sidebar.file_uploader("Or Upload a Video File", type=["mp4", "avi", "mkv"])

if st.sidebar.button("Process Video"):
    overall_start_time = time.time()
    with st.spinner("Processing... ⏳"):
        if video_url:
            st.info("Downloading video...")
            video_file = download_video(video_url)
        elif uploaded_file:
            video_file = os.path.join(TEMP_DIR, uploaded_file.name)
            with open(video_file, "wb") as f:
                f.write(uploaded_file.read())
        else:
            st.error("Please enter a video URL or upload a file.")
            st.stop()

        st.info("Transcribing and summarizing...")
        transcript, summary, transcription_time, summarization_time, audio_extraction_time = transcribe_and_summarize(video_file)
        overall_end_time = time.time()

        st.subheader("📜 Transcription:")
        st.text_area("", transcript, height=200)

        st.subheader("📄 Summary:")
        st.write(summary)

        st.subheader("⏳ Processing Time:")
        st.write(f"📝 **Transcription Time:** {transcription_time:.2f} seconds")
        st.write(f"📄 **Summarization Time:** {summarization_time:.2f} seconds")
        st.write(f"🎵 **Audio Extraction Time:** {audio_extraction_time:.2f} seconds")
        st.write(f"🚀 **Total Processing Time:** {(overall_end_time - overall_start_time):.2f} seconds")

        st.success("✅ Done!")


2025-04-02 05:24:46.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 05:24:46.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 05:24:46.077 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 05:24:46.079 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 05:24:46.080 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 05:24:46.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 05:24:46.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 05:24:46.083 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar